# 1. Inicjalizacja API i załadowanie danych
Celem inicjalizacji wykorzystujemy klasę CloudberryConfig, jako argument podając adres serwera cloudberry-cb.

In [ ]:
# Define Cloudberry configuration

import cloudberry.api as cb
cb_port = 9000
cb_config = cb.CloudberryConfig(f'http://localhost:{cb_port}')

Następnie wgrywamy przykładowe pliki z eksperymentów przeprowadzonych na platformie AgE. Wykorzystujemy w tym celu AgeFileUploader. Metoda upload_file zwraca informacje o wczytanych z pliku danych dotyczących uruchomienia, którą zapisujemy.

In [ ]:
# Initialize API for data upload

cb_uploader = cb.AgeFileUploader(cb_config)

files_paths = [
    "./data/emas-20190411T232808.log",
    "./data/emas-20190411T234810.log",
    "./data/emas-20190412T000813.log",
    "./data/emas-20190412T120536.log",
]

files_details = cb.AgeUploadDetails(
    headers_keys={"[WH]": "[W]", "[SH]": "[S]", "[BH]": "[B]"}
)

experiment_name = 'Example AgE experiment'

uploaded_computations = []
for fp in files_paths:
    computation = cb_uploader.upload_file(file_path=fp, experiment_name=experiment_name, details=files_details)
    uploaded_computations.append(computation)

## 1.1. API do analizy poszczególnych uruchomień

Poniżej inicjalizujemy kolejny moduł API - ComputationSeries.
Metoda for_computations pobiera dla wybranych uruchomień szczegółowe dane danego uruchomienia, dla wybranego parametru.

Każda metoda API oprócz serii generuje również serię średnią, wraz z odchyleniem standardowym. Zwracany obiekt to DataSeriesPack, który ma następujące pola:
- series - lista obiektów DataSeries, każdy reprezentuje dane z 1 uruchomienia
- average_series - obiekt DataSeries, reprezentujący serię średnią


In [ ]:
# Compare uploaded computations and compute average and std

cb_computations = cb.ComputationSeries(cb_config)

field = 'BEST_SOLUTION_SO_FAR'
series_pack = cb_computations.for_computations(uploaded_computations, field)

Możemy podejrzeć zawartość zwróconych serii, konwertując je do DataFrame z biblioteki pandas.

In [ ]:
# Preview returned series as data frames

# series_pack.series[0].as_data_frame
series_pack.average_series.as_data_frame

Aby nanieść pobrane dane na wykres, należy użyć SeriesConverter i przekonwertować model danych z DataSeriesPack do PlotSeriesPack:

In [ ]:
# Perform conversion to plots model

from cloudberry.converters import SeriesConverter

plot_pack = SeriesConverter.data_series_pack_to_plot_series_pack(
    ds=series_pack,
    x_field="_time",
    y_field=field,
    y_err_field="stddev"
)

Konieczne jest jeszcze zainicjalizowanie API do tworzenia wykresów, poprzez utworzenie obiektu PlotProperties. Możemy modyfikować pola tego obiektu, aby konfigurować finalny wygląd wykresów.

In [ ]:
# Configure cloudberry.plots

import cloudberry.plots as cp

props = cp.PlotProperties.default()
props.title = "Best fitness for uploaded computations"
props.show_title = True
props.x_axis_name = "Time in milliseconds"
props.y_axis_name = "Best fitness"
# props.show_series = False
# props.show_error_bars = True
props.default_series_kind = cp.PlotSeriesKind.LINE

Chcąc utworzyć nowy wykres korzystamy z klasy PlotBuilder, którą inicjalizujemy utworzonymi wcześniej PlotProperties.

Serie możemy dodawać przy pomocy metod:
* add_pack - dodaje PlotSeriesPack
* add_series - dodaje pojedynczą PlotSeries
* add_avg_series - dodaje pojedynczą PlotSeries jako serię średnią
* add_trend - dodaje linie trendu

In [ ]:
builder = cp.PlotBuilder(props)
builder.add_pack(plot_pack)
plot = builder.plot()
plot

Utworzony wykres możemy wyeksportować do wybranego formatu: png, svg lub jpeg z dowolną rozdzielczością, modyfikując parametry metody.

In [ ]:
# Optionally export plot to file
cp.PlotlyExporter(plot).write_image(file='best_solution_computations.png', scale=4)

## 1.2. API do analizy użytych konfiguracji
Aby pobrać informacje o konifiguracjach zastosowanych w zaimportowanych plikach, należy użyć API klasy Metadata. 

In [ ]:
# Check what configuration were used in uploaded files
cb_meta = cb.Metadata(cb_config)
uploaded_configurations = list(cb_meta.experiment_configuration_api().find_by_id(id) for id in set([c.configuration_id for c in uploaded_computations]))

Do właściwej analizy wykorzystujemy API klasy ConfigurationSeries.
Poniżej postępujemy analogicznie jak w przypadku analizy uruchomień, tworząc nowy wykres.

In [ ]:
# Compare averages for configuration

cb_configurations = cb.ConfigurationSeries(cb_config)
series_pack = cb_configurations.for_configurations(uploaded_configurations, field)
plot_pack = SeriesConverter.data_series_pack_to_plot_series_pack(
    ds=series_pack,
    x_field="_time",
    y_field=field,
    y_err_field="stddev"
)

props.title = "Best fitness for configurations"
# props.show_error_bars = True
builder = cp.PlotBuilder(props)
builder.add_pack(plot_pack)

# Do not show error bars on 'average' series
for s in plot_pack.series:
    s.y_err_field = None

plot = builder.plot()
plot


In [ ]:
plot_pack.series

In [ ]:
# export plot PNG
cp.PlotlyExporter(plot).write_image(file='best_solution_configurations.png', scale=4)

Poniżej usuwamy zaimportowane dane.

In [ ]:
# Cleanup
cb_deletion = cb.Deletion(cb_config)
cb_deletion.delete_computations(uploaded_computations)
cb_deletion.delete_configurations(uploaded_configurations)

# Verify cleanup
cb_meta.experiment_computation_api().find_all()

# Warning: Total cleanup
# v = cb_meta.experiment_computation_api().find_all()
# cb_deletion.delete_computations(v)